In [116]:
%load_ext autoreload
%autoreload 2

from lunar_lander import LunarLander
from gym.wrappers.monitoring.video_recorder import VideoRecorder
import torch
import numpy as np

from solution import MLPCritic, MLPActorCritic, MLPCategoricalActor
from solution import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
env = LunarLander()


rec = VideoRecorder(env, "policy.mp4")
episode_length = 300
n_eval = 100
returns = []
print("Evaluating agent...")

Evaluating agent...


In [17]:
state = env.reset()
state_tensor = torch.tensor(state)
state_tensor.shape

torch.Size([8])

In [41]:
ac = MLPActorCritic()

a, v, logp_a = ac.step(state_tensor)


In [58]:
obs_dim = [8]
act_dim = []

# Training parameters
# You may wish to change the following settings for the buffer and training
# Number of training steps per epoch
steps_per_epoch = 3000
# Number of epochs to train for
epochs = 50
# The longest an episode can go on before cutting it off
max_ep_len = 300
# Discount factor for weighting future rewards
gamma = 0.99
lam = 0.97

In [138]:
# Set up buffer
buf = VPGBuffer(obs_dim, act_dim, steps_per_epoch, gamma, lam)

state, ep_ret, ep_len = env.reset(), 0, 0
a, v, logp = ac.step(torch.as_tensor(state, dtype=torch.float32))
print(a)
next_state, r, terminal = env.transition(int(a))
# Log transition
buf.store(state, a, r, v, logp)
buf.store(state, a, r, v, logp)
buf.store(state, a, r, v, logp)

tensor(0)


In [115]:
buf.end_traj()

In [139]:
np.ones(buf.ptr-buf.path_start_idx)

array([1., 1., 1.])

In [140]:
self = buf
path_slice = slice(self.path_start_idx, self.ptr)
rews = np.append(self.rew_buf[path_slice], 0)
vals = np.append(self.val_buf[path_slice], 0)

rews

array([5.67799854, 5.67799854, 5.67799854, 0.        ])

In [143]:
discount_cumsum(rews, gamma)


array([16.86422347, 11.2992171 ,  5.67799854,  0.        ])

In [136]:
discount_cumsum(rews[:self.ptr], self.gamma).shape

(3,)